In [ ]:
!pip install -e /workspace/src/datetime_matcher
# !pip install git+https://github.com/jedzill4/datetime_matcher

In [ ]:
!pip uninstall -y datetime-matcher

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport datetime_matcher.model_types
%aimport datetime_matcher.datetime_extractor
%aimport datetime_matcher.regex_generator
%aimport datetime_matcher.dfregex_lexer
%aimport datetime_matcher.datetime_matcher

In [ ]:
import locale
from datetime_matcher import DatetimeMatcher

dtm = DatetimeMatcher()
locale.setlocale(locale.LC_ALL, "es_AR.UTF-8")


In [ ]:
list(dtm.extract_datetimes('%d/%m/%Y', '1/11/2015'))

In [ ]:

dtm.sub(r'%d/%m/%Y', r'%-d/%B/%Y', '01/11/2015')

In [ ]:

dtm.sub(r'%-d/%m/%Y', r'%d/%b/%y %-H:%-M', '1/11/2015')

In [ ]:
import re
import pandas as pd
from datetime import datetime
from datetime_matcher.dfregex_lexer import DfregexLexer
from datetime_matcher.regex_generator import RegexGenerator
from datetime_matcher.datetime_extractor import DatetimeExtractor

tokenizer = DfregexLexer()
regex_gen = RegexGenerator()
extractor = DatetimeExtractor()

text = "1/11/2045"
fmt = "%-d/%m/%Y"

tokens = list(tokenizer.tokenize(fmt))
regex = regex_gen.generate_regex(tokens, True)

# results = list(extractor.extract_datetimes(regex, tokens, text))
df_tokens = list(token for token in tokens if token.kind == "DATETIME_FORMAT_CODE")
matches = list(extractor._DatetimeExtractor__finditer_with_limit(regex, text, 0))

raw_regex = re.findall(regex, text)
print("raw regex:", raw_regex)

print("tokens:", tokens)
print("regex:", regex)
# print("result:", results)
print('results:')
print('df_tokens:', df_tokens)
print('matches', matches)

for match in matches:
    # print(extractor._DatetimeExtractor__parse_match_into_maybe_datetime(match, df_tokens))
    datetime_format_codes = []
    datetime_string_values = []
    for group_key, group_value in match.groupdict().items():
        print(group_key, group_value)
        # Find only the df groups and match up the format codes with the actual datetime values
        if group_key.startswith("DF___"):
            try:
                datetime_group_num = int(group_key[5:])
                datetime_format_codes.append(df_tokens[datetime_group_num].value)
                datetime_string_values.append(group_value)
            except Exception:
                # Skip all problematic ones
                continue
    # Now construct strings to use for strptime to generate a datetime object from the values
    datetime_formatter = "#".join(datetime_format_codes).replace('%-', '%')
    datetime_string = "#".join(datetime_string_values)

    try:
        parsed_datetime = datetime.strptime(datetime_string, datetime_formatter)
    except ValueError as err:
        # if "'-' is a bad directive in format" in err:
            # parsed_datetime = datetime.strptime(datetime_string, datetime_formatter.replace('%-', '%'))
        # If there is a problem, we still need to return a value to maintain
        # one-to-one mapping between regex match and datetime
        parsed_datetime = None
    print(datetime_formatter)
    print(datetime_string)
    parsed_datetime = datetime.strptime(datetime_string, datetime_formatter)
    # parsed_datetime = pd.to_datetime(datetime_string, format=datetime_formatter)
    print(parsed_datetime)


In [ ]:
import sys
sys.version_info >= (3, 7)